In [22]:
!pip install yelpapi

In [26]:
from y_config import api_key 
import yelpapi
import json
import requests
from  pprint import  pprint
import pandas as pd
import numpy as np
import random

counties=pd.read_csv("acs2015_county_data.csv")
counties["county_string"] = counties["County"] + " County, " + counties["State"]
listofcounties = counties["county_string"].tolist()

In [27]:
zipcodes=pd.read_csv("zip_codes_states.csv")
zipcodes["county_string"] = zipcodes["county"] + " County, " + zipcodes["state"]
listofzipcodes=zipcodes["county_string"].unique().tolist()

In [28]:
#county list and zip list have discrepancy. need to pick counties that have entries in both lists

fullcountylist = list(set(listofcounties) & set(listofzipcodes))
sample_list = random.sample(fullcountylist,50)

In [29]:
#merged_df = zipcodes.merge(counties, left_on="county_string", right_on="county_string", how="left")
#maybe this wan't necessary but it's here if we need it
res_in_county = pd.DataFrame(sample_list)
res_in_county.columns = ["county_string"]
res_in_county["restaurant count"]=0
res_in_county["reviews"]=0
res_in_county["avg score"]=0
res_in_county["num of Italian"]=0
res_in_county["num of Mexican"]=0
res_in_county["num of Chinese"]=0
res_in_county["num of Indian"]=0
res_in_county.head()



,county_string,restaurant count,reviews,avg score,num of Italian,num of Mexican,num of Chinese,num of Indian
0,"Scott County, Indiana",0,0,0,0,0,0,0
1,"Desha County, Arkansas",0,0,0,0,0,0,0
2,"Douglas County, Illinois",0,0,0,0,0,0,0
3,"Madison County, Ohio",0,0,0,0,0,0,0
4,"Sanders County, Montana",0,0,0,0,0,0,0


In [30]:
res_df = pd.DataFrame(columns=["name","zipcode","county_string","rating","review_count","category","transaction type"])
res_df

,name,zipcode,county_string,rating,review_count,category,transaction type


In [32]:
r = 0
for c in sample_list:
    county_string = str(c)
    for term in ["Italian","Mexican","Chinese","Indian"]:
        #we are allowed 5000 queries per 24 hours. if we run out, we can create another. till then, use this
        yelpkey="C8I4eLOGfKC0Ni9OwrjQXNjQW5MHDwB34C390sRP5KLXpEwWg7Dbk-Sbq5REVgF19GtJxe-oe2xciOGfouLPahK4O9kArrG40ZsxXOKyaZ1u-NeU7RGgeOipBf7DWnYx"
        response = requests.get(url="https://api.yelp.com/v3/transactions/{transaction_type}/search", 
                                params= {"location": county_string, 
                                         "term": term, "limit":"50"}, 
                                headers={'Authorization': 'bearer %s' % yelpkey})
        responsejson = response.json()
        
        try:
            count=0
            
            for x in range(0,49):
                for y in range(0,len(responsejson["businesses"][x]["categories"])):
                    if ((responsejson["businesses"][x]["categories"][y]["title"]==term) and 
                        (responsejson["businesses"][x]["review_count"] > 0)):
                        
                        zc = int(responsejson["businesses"][x]["location"]["zip_code"])
                        if (zipcodes.loc[zipcodes["zip_code"]==zc]["county_string"].tolist()[0]== county_string):
                            
                            print("\n",responsejson["businesses"][x]["name"],
                                  responsejson["businesses"][x]["location"]["zip_code"],
                                  zipcodes.loc[zipcodes["zip_code"]==zc]["county_string"].tolist()[0],
                                  responsejson["businesses"][x]["rating"],
                                  responsejson["businesses"][x]["review_count"],
                                  responsejson["businesses"][x]["categories"][y]["title"])
                            
                            res_df.loc[r]=[responsejson["businesses"][x]["name"],
                                  zc,
                                  county_string,
                                  responsejson["businesses"][x]["rating"],
                                  responsejson["businesses"][x]["review_count"],
                                  term]
                            count += 1
                            r += 1
        except:
            print("")
    
        print("End of list. Found", count, term, "restaurants in", county_string)
  



End of list. Found 0 Italian restaurants in Scott County, Indiana

End of list. Found 0 Mexican restaurants in Scott County, Indiana

End of list. Found 0 Chinese restaurants in Scott County, Indiana

End of list. Found 0 Indian restaurants in Scott County, Indiana

End of list. Found 0 Italian restaurants in Desha County, Arkansas

End of list. Found 0 Mexican restaurants in Desha County, Arkansas

End of list. Found 0 Chinese restaurants in Desha County, Arkansas

End of list. Found 0 Indian restaurants in Desha County, Arkansas

End of list. Found 0 Italian restaurants in Douglas County, Illinois

End of list. Found 0 Mexican restaurants in Douglas County, Illinois

End of list. Found 0 Chinese restaurants in Douglas County, Illinois

End of list. Found 0 Indian restaurants in Douglas County, Illinois

End of list. Found 0 Italian restaurants in Madison County, Ohio

End of list. Found 0 Mexican restaurants in Madison County, Ohio

End of list. Found 0 Chinese restaurants in Madiso


End of list. Found 0 Chinese restaurants in Hendricks County, Indiana

End of list. Found 0 Indian restaurants in Hendricks County, Indiana

End of list. Found 0 Italian restaurants in Mahaska County, Iowa

End of list. Found 0 Mexican restaurants in Mahaska County, Iowa

End of list. Found 0 Chinese restaurants in Mahaska County, Iowa

End of list. Found 0 Indian restaurants in Mahaska County, Iowa

End of list. Found 0 Italian restaurants in Columbia County, Wisconsin

End of list. Found 0 Mexican restaurants in Columbia County, Wisconsin

End of list. Found 0 Chinese restaurants in Columbia County, Wisconsin

End of list. Found 0 Indian restaurants in Columbia County, Wisconsin

End of list. Found 0 Italian restaurants in Sullivan County, Pennsylvania

End of list. Found 0 Mexican restaurants in Sullivan County, Pennsylvania

End of list. Found 0 Chinese restaurants in Sullivan County, Pennsylvania

End of list. Found 0 Indian restaurants in Sullivan County, Pennsylvania

End of li

In [13]:
res_df

,name,zipcode,county_string,rating,review_count,category,transaction type


In [25]:
response.json("transaction")

NameError: name 'response' is not defined

In [20]:
#we are allowed 5000 queries per 24 hours. if we run out, we can create another. till then, use this
api_key="C8I4eLOGfKC0Ni9OwrjQXNjQW5MHDwB34C390sRP5KLXpEwWg7Dbk-Sbq5REVgF19GtJxe-oe2xciOGfouLPahK4O9kArrG40ZsxXOKyaZ1u-NeU7RGgeOipBf7DWnYx"
response = requests.get(url="https://api.yelp.com/v3/transactions/{transaction_type}/search", 
                        params= {"location": county_string, "term": term, "limit":"50"}, 
                        headers={'Authorization': 'bearer %s' % yelpkey})
        
        

NameError: name 'county_string' is not defined

In [ ]:
trans =["pickup", "delivery", "restaurant_reservation" ]:

In [24]:
###EXAMPLE

#Transaction Search API: https://www.yelp.com/developers/documentation/v3/transactions_search
\n{}\n.format("yelp_api.transaction_search_(query(transaction_type= ("delivery", "restaurant_reservation"), 
                                                   location=res_df[county_string],)))
response = yelp_api.transaction_search_query(transaction_type='delivery', location='________')
pprint(response)


SyntaxError: unexpected character after line continuation character (<ipython-input-24-16ca18f22084>, line 5)

In [ ]:
def parse(url):
    # url = "https://www.yelp.com/biz/frances-san-francisco"
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.90 Safari/537.36'}
    response = requests.get(url, headers=headers, verify=False).text
    parser = html.fromstring(response)
    print "Parsing the page"